In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Irina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#visualization
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [6]:
nlp = spacy.load('nl_core_news_lg')

In [7]:
def load_data(file):
    with open (file, "r", encoding="latin-1") as f:
        data = f.read() 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [8]:
stopwords = stopwords.words("dutch")

In [17]:
import os
import re

texts = []

#for file in os.listdir("data"):
for file in os.listdir("Summarization/summaries_txt"):
        if re.match(r"\d*\.txt", file):
            with open(f"Summarization/summaries_txt/{file}", 'r', encoding="latin-1") as text:
            # with open(f"data/{file}", 'r', encoding="utf-8") as text:
                content = text.read()
                content = content.encode("utf-8", "ignore")
                content = content.decode()
                content = content.encode('utf-8').decode('utf-8')
                listCont = list(content.split(" "))
                # print(listCont)
                no_duplicates = []
                #for i in listCont:
                #    if listCont[i]!=listCont[i-1]:
                #        no_duplicates.append(i)
                #        print(no_duplicates)
                no_duplicates.append(listCont[0])
                [no_duplicates.append(x) for x in listCont if x != no_duplicates[-1]]
                print(no_duplicates)
                content = ' '.join(no_duplicates)
                texts.append(content)


print(len(texts))
print(texts[0])

['De', 'Wet', 'tot', 'wijziging', 'van', 'de', 'Overeenkomst', 'tussen', 'het', 'Koninkrijk', 'België', 'en', 'de', 'Staat', 'Malta', 'tot', 'het', 'vermijden', 'van', 'dubbele', 'belasting', 'en', 'tot', 'het', 'voorkomen', 'van', 'het', 'ontgaan', 'van', 'belasting,', 'zoals', 'gewijzigd', 'door', 'de', 'Aanvullende', 'Overeenkomst', 'ondertekend', 'te', 'Brussel', 'op', '23', 'juni', '1993,', 'zal', 'volkomengevolg', 'hebben.']
['De', 'Wet', 'tot', 'wijziging', 'van', 'de', 'Overeenkomst', 'tussen', 'het', 'Koninkrijk', 'België', 'en', 'de', 'Staat', 'Malta', 'tot', 'het', 'vermijden', 'van', 'dubbele', 'belasting', 'en', 'tot', 'het', 'voorkomen', 'van', 'het', 'ontgaan', 'van', 'belasting,', 'zoals', 'gewijzigd', 'door', 'de', 'Aanvullende', 'Overeenkomst', 'ondertekend', 'te', 'Brussel', 'op', '23', 'juni', '1993,', 'zal', 'volkomengevolg', 'hebben.']
['Het', 'ministerie', 'van', 'Financiën', 'en', 'de', 'staatssecretaris', 'van', 'Financiën', 'heeft', 'een', 'nieuwe', 'werkwijze

In [18]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("nl_core_news_lg", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(texts)
print (lemmatized_texts)

['wet wijziging overeenkomst staat vermijden dubbel belasting voorkomen ontgaan belasting wijzigen Aanv_ullend overeenkomst ondertekenen hebben', 'ministerie Financiën staatssecretaris financiën hebben nieuw werkwijze aan_kondigen akt akte attest erf_opvolging aan_stellen nieuw werkwijze omvatten akt akte attest erfop_volging ontvanger notaris', 'koninklijk besluit wijziging koninkl_ijkbesluit vaststelling tarief belasting toegevoegde waarde indeling goed dienst tarief bepalen product bestemmen intiem hygiënisch bescherming extern defibrillator betreffen bekrachtigen wet Belgisch Staatsblad', 'demissionair minister Brussels hoofdstad verantwoordelijk Financiën begroting voorlopig regeling treffen voorwaarde behoud fiscaal gunstregime bekomen kader overdracht eenfamiliaal onderneming familiaal vennootschap verlengen', 'ministerie financiën hebben nieuw aanvraag in_dienen belas_tingvermindering verwerving nieuw aandeel startende onderneming artikel WIB', 'nieuw afdeling hoog onderwijs so

In [19]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['wet', 'wijziging', 'overeenkomst', 'staat', 'vermijden', 'dubbel', 'belasting', 'voorkomen', 'ontgaan', 'belasting', 'wijzigen', 'aanv_ullend', 'overeenkomst', 'ondertekenen', 'hebben']


In [25]:
print(type(data_words))

<class 'list'>


# bigrams, trigrams

In [20]:
bigrams_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=30)
trigrams_phrases = gensim.models.Phrases(bigrams_phrases[data_words], threshold=30)

bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigrams_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])
    
def make_trigrams(texts):
    return([trigram[bigram[doc]] for doc in texts])


In [21]:

data_bigrams = make_bigrams(data_words)

data_bigrams_trigrams = make_bigrams(data_bigrams)

print(data_bigrams_trigrams)

[['wet', 'wijziging', 'overeenkomst', 'staat', 'vermijden', 'dubbel', 'belasting', 'voorkomen', 'ontgaan', 'belasting', 'wijzigen', 'aanv_ullend', 'overeenkomst', 'ondertekenen', 'hebben'], ['ministerie', 'financien', 'financien', 'hebben', 'nieuw', 'werkwijze', 'aan_kondigen', 'akt', 'akte', 'attest', 'erf_opvolging', 'aan_stellen', 'nieuw', 'werkwijze', 'omvatten', 'akt', 'akte', 'attest', 'erfop_volging', 'ontvanger', 'notaris'], ['koninklijk', 'besluit', 'wijziging', 'vaststelling', 'tarief', 'belasting_toegevoegde', 'waarde', 'indeling', 'goed', 'dienst', 'tarief', 'bepalen', 'product', 'bestemmen', 'intiem', 'hygienisch', 'bescherming', 'extern', 'defibrillator', 'betreffen', 'bekrachtigen', 'wet', 'belgisch_staatsblad'], ['demissionair_minister', 'brussels', 'hoofdstad', 'financien', 'begroting', 'voorlopig', 'regeling', 'treffen', 'voorwaarde', 'behoud', 'fiscaal', 'gunstregime', 'bekomen', 'kader', 'overdracht', 'eenfamiliaal', 'onderneming', 'familiaal', 'vennootschap', 'verl

In [22]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
#print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [39]:
'''
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)
'''

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]
aanv_ullend


In [23]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=17,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

# Visualize

In [24]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis


c:\Users\Irina\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.443737  0.108054       1        1  19.214849
1      0.104416 -0.418630       2        1  16.407530
4     -0.343649 -0.314843       3        1  12.189561
0     -0.147906  0.412301       4        1  10.274599
2      0.281081 -0.264044       5        1   9.422126
10     0.232129 -0.110147       6        1   5.141632
12     0.151954  0.319161       7        1   4.817417
11    -0.073160 -0.252149       8        1   4.292932
7     -0.328789  0.106423       9        1   4.147056
3      0.068831  0.159007      10        1   3.134178
8      0.195329  0.088501      11        1   2.607850
5     -0.077584  0.173242      12        1   2.313173
15    -0.226254 -0.053118      13        1   2.163244
13    -0.160617  0.088965      14        1   1.476546
14    -0.038767 -0.060136      15        1   1.204479
16    -0.063613  0.007256      16        1   0.647422
6     -0.017139  0.010158      17        1   0.545407, topic_info=            Term        Freq       Total Category  logprob  loglift
32       besluit  396.000000  396.000000  Default  30.0000  30.0000
116    maatregel  203.000000  203.000000  Default  29.0000  29.0000
12     wijziging  167.000000  167.000000  Default  28.0000  28.0000
3         hebben  218.000000  218.000000  Default  27.0000  27.0000
21    ministerie  194.000000  194.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
1391      sexies    0.022970    0.983200  Topic17  -7.8514   1.4548
778       bedrag    0.022978    0.987154  Topic17  -7.8511   1.4511
69     in_dienen    0.022974   67.351863  Topic17  -7.8512  -2.7719
929      omwille    0.022970    0.985167  Topic17  -7.8514   1.4528
1387         bis    0.022970    0.983738  Topic17  -7.8514   1.4542

[614 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
13        1  0.940787  aan_kondigen
281       2  0.989509     aan_nemen
134       7  0.714004    aan_pakken
201       3  0.920406    aan_vullen
469      11  0.589486    aan_wijzen
...     ...       ...           ...
451       5  0.857495          zoek
143       7  0.819196          zorg
411       9  0.855824   zorg_sector
482       8  0.476970        zoveel
244      15  0.548651       zwaarst

[480 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 2, 5, 1, 3, 11, 13, 12, 8, 4, 9, 6, 16, 14, 15, 17, 7])